In [ ]:
#!/usr/bin/env python3
# LSTM(SE) - Hypertuned, audit-ready, pure-sign trading, finalisation pack

import os, sys, json, math, time, random, hashlib, platform, warnings, zipfile
from pathlib import Path
from datetime import datetime
from typing import List, Tuple, Dict, Any, Optional

# ---- CUDA determinism env must be set BEFORE any torch CUDA checks ----
os.environ.setdefault("CUBLAS_WORKSPACE_CONFIG", ":4096:8")  # ensures deterministic GEMMs (where supported)

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# -------------------- determinism --------------------
RANDOM_SEED = 1337
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
try:
    torch.use_deterministic_algorithms(True)
except Exception:
    pass
torch.set_num_threads(1)  # optional stricter reproducibility

# -------------------- config --------------------
CONFIG = {
    "MODEL_ID": "LSTM_SE_TUNED",
    "REPORTED_NAME": "LSTM (sentiment-enhanced) - monthly refit",
    "TICKERS": ["AAPL","AMZN","MSFT","TSLA","AMD"],
    "INPUT_DIR": "final_inputs",
    "OUT_ROOT": "LSTM_SE_TUNED_FINAL",

    # Fixed splits
    "TRAIN_START": "2021-02-03",
    "TRAIN_END":   "2022-12-30",
    "VAL_START":   "2023-01-03",
    "VAL_END":     "2023-05-31",
    "TEST_START":  "2023-06-01",
    "TEST_END":    "2023-12-28",
    "ASSERT_N_TEST": 146,

    # Modelling
    "SEQ_LEN": 90,
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    "EPS_DA": 0.0010,      # for Directional Accuracy only (not used for trading)
    "EPOCHS": 500,

    # Lean, defensible Val-only search space (per ticker)
    "SEARCH_SPACE": {
        "hidden_size": [32, 48, 64, 96],
        "num_layers": [1, 2],
        "dropout": [0.0, 0.1, 0.2],
        "optim": ["Adam", "AdamW"],
        "lr": [1e-4, 3e-4, 1e-3],
        "batch_size": [32, 64],
        "patience": [10, 15],
        "weight_decay": [0.0, 1e-4],
        "grad_clip_norm": [None, 1.0]
    },
    "N_TRIALS": 24,

    # Global invariants (clock/target provenance)
    "CLOCK_INVARIANTS": {
        "market_timezone": "America/New_York",
        "cutoff_local_time": "16:00:00",
        "no_forward_fill_past_cutoff": True,
        "sentiment_zero_encoding_on_no_activity_days": True
    },

    # Code provenance (adapted example)
    "PROVENANCE": {
        "repo_name": "keras-team/keras-io",
        "repo_url": "https://github.com/keras-team/keras-io",
        "licence": "Apache-2.0",
        "commit_sha": "f7e54711205ca29a06577a04047b4a40df32fdec",
        "imported_files": "examples/timeseries/timeseries_weather_forecasting.py",
        "adaptation_notes": "Adapted stacked-LSTM; project loaders; zero-preserving scaler; monthly refit; EMR metrics."
    },

    # Optional peer features manifest for parity when packs are not present
    "PEER_FEATURES_FILE": "peer_features_manifest.json"
}

# -------------------- IO --------------------
def read_csv(ticker: str) -> pd.DataFrame:
    p = Path(CONFIG["INPUT_DIR"]) / f"{ticker}_input.csv"
    if not p.exists():
        raise FileNotFoundError(f"Missing input file: {p}")
    df = pd.read_csv(p)
    if "date" not in df.columns or "Target" not in df.columns:
        raise RuntimeError(f"{p} must include 'date' and 'Target'")
    df["date"] = pd.to_datetime(df["date"])
    if df["date"].duplicated().any():
        dups = df[df["date"].duplicated()]["date"].unique()[:5]
        raise RuntimeError(f"{p} has duplicate dates (e.g. {dups})")
    df = df.sort_values("date").reset_index(drop=True)
    if df["Target"].isna().any():
        raise RuntimeError(f"{p} has NaNs in Target")
    if "Close" in df and df["Close"].isna().any():
        raise RuntimeError(f"{p} has NaNs in Close")
    return df

def slice_dates(df: pd.DataFrame, a: str, b: str) -> pd.DataFrame:
    m = (df["date"] >= pd.to_datetime(a)) & (df["date"] <= pd.to_datetime(b))
    return df.loc[m].reset_index(drop=True)

def group_cols(cols: List[str]) -> Tuple[List[str], List[str]]:
    sents = [c for c in cols if c.startswith(("Tw_", "Rd_", "Nw_SP500_"))]
    base  = [c for c in cols if c not in sents and c not in ["date", "ticker", "Target"]]
    return base, sents

# -------------------- scalers --------------------
class ZeroPreservingStandardiser:
    def __init__(self, sent_cols: List[str]):
        self.sent_cols = set(sent_cols)
        self.mu, self.sd = {}, {}
        self.fitted = False
    def fit(self, X: pd.DataFrame):
        for c in X.columns:
            if c in self.sent_cols:
                v = X[c].to_numpy()
                nz = v[v != 0]
                mu = float(np.mean(nz)) if len(nz) else 0.0
                sd = float(np.std(nz)) if len(nz) else 1.0
                self.mu[c] = mu
                self.sd[c] = sd if sd > 1e-12 else 1.0
            else:
                mu = float(np.mean(X[c])); sd = float(np.std(X[c]))
                self.mu[c] = mu; self.sd[c] = sd if sd > 1e-12 else 1.0
        self.fitted = True
        return self
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        if not self.fitted:
            raise RuntimeError("Scaler not fitted")
        Z = X.copy()
        for c in X.columns:
            if c in self.sent_cols:
                v = Z[c].to_numpy(dtype=float)
                mask = (v != 0)
                v2 = v.copy()
                v2[mask] = (v2[mask] - self.mu[c]) / self.sd[c]
                v2[~mask] = 0.0
                Z[c] = v2
            else:
                Z[c] = (Z[c] - self.mu[c]) / self.sd[c]
        return Z

def zero_preservation_report(raw_df: pd.DataFrame, scaled_df: pd.DataFrame, sent_cols: List[str], out_csv: Path, atol: float = 1e-12):
    rows, violations = [], 0
    if not sent_cols:
        pd.DataFrame([{"column":"<none>","raw_zero_count":0,"preserved_zero_count":0,"violations":0,"ok":True,"zeros_after_total":0}]).to_csv(out_csv, index=False)
        return
    for c in sent_cols:
        raw = raw_df[c].to_numpy()
        if not np.all(np.isfinite(raw)):
            raise RuntimeError(f"Non-finite values detected in sentiment column '{c}'")
        sca = scaled_df[c].to_numpy()
        mask_raw_zero = np.isfinite(raw) & (raw == 0)
        n_raw_zeros = int(mask_raw_zero.sum())
        n_preserved = int(np.isclose(sca[mask_raw_zero], 0.0, atol=atol, rtol=0).sum())
        v = max(n_raw_zeros - n_preserved, 0)
        violations += v
        rows.append({
            "column": c,
            "raw_zero_count": n_raw_zeros,
            "preserved_zero_count": n_preserved,
            "violations": v,
            "ok": (v == 0),
            "zeros_after_total": int(np.isclose(sca, 0.0, atol=atol, rtol=0).sum())
        })
    pd.DataFrame(rows).to_csv(out_csv, index=False)
    if violations > 0:
        raise RuntimeError(f"Zero-preservation check failed: {violations} violations")

# -------------------- datasets / helpers --------------------
class SeqDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32); self.y = y.astype(np.float32)
    def __len__(self): return self.X.shape[0]
    def __getitem__(self, i): return self.X[i], self.y[i]

def windowise(df: pd.DataFrame, feat_cols: List[str], L: int):
    n = len(df)
    if n <= L:
        raise ValueError(f"Not enough rows ({n}) to form windows with seq_len={L}")
    X = df[feat_cols].to_numpy(); y = df["Target"].to_numpy()
    xs, ys = [], []
    for i in range(n - L):
        xs.append(X[i:i+L, :]); ys.append(y[i+L])
    return np.stack(xs).astype(np.float32), np.array(ys, dtype=np.float32)

def naive_last_close_from_windowised(close_series: pd.Series, L: int) -> np.ndarray:
    """
    Given the same df used for windowise (length N), the targets are y[L:].
    For each target at t>=L, return Close_{t-1}. Length is N-L.
    """
    c = close_series.to_numpy()
    return c[L-1:-1]  # indices [L-1 .. N-2] -> length N-L

def feature_stable_set(train_by_t: Dict[str, pd.DataFrame], val_by_t: Dict[str, pd.DataFrame]) -> List[str]:
    first_t = next(iter(train_by_t.keys()))
    base0, sents0 = group_cols(list(train_by_t[first_t].columns))
    feats0 = [c for c in base0 + sents0 if c not in ["date", "ticker", "Target"]]
    tr0, va0 = train_by_t[first_t], val_by_t[first_t]
    keep = {c for c in feats0 if float(tr0[c].std()) > 0 and float(va0[c].std()) > 0}
    for t in train_by_t:
        tr, va = train_by_t[t], val_by_t[t]
        base, sents = group_cols(list(tr.columns))
        feats = [c for c in base + sents if c not in ["date", "ticker", "Target"]]
        ok = {c for c in feats if float(tr[c].std()) > 0 and float(va[c].std()) > 0}
        keep = keep & ok
    out = sorted(list(keep))
    if "Close" in feats0 and "Close" not in out:
        out = sorted(list(set(out + ["Close"])))
    return out

def monthly_boundaries(te_block: pd.DataFrame) -> List[int]:
    d = te_block.copy()
    ym = d["date"].dt.to_period("M")
    starts = (ym != ym.shift()).to_numpy()
    return np.flatnonzero(starts).tolist()  # positional offsets

# -------------------- model --------------------
class LSTMReg(nn.Module):
    def __init__(self, n_in, hidden, layers, dropout):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_in, hidden_size=hidden, num_layers=layers,
                            batch_first=True, dropout=(dropout if layers > 1 else 0.0))
        self.head = nn.Linear(hidden, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        h = out[:, -1, :]
        y = self.head(h).squeeze(-1)
        return y

def build_loader(X, y, bs):
    return DataLoader(SeqDataset(X, y), batch_size=bs, shuffle=False,
                      num_workers=0, pin_memory=torch.cuda.is_available(), drop_last=False)

def train_one(model, train_loader, val_loader, cfg):
    dev = CONFIG["DEVICE"]
    model = model.to(dev)
    if cfg["optim"] == "Adam":
        opt = torch.optim.Adam(model.parameters(), lr=cfg["lr"], weight_decay=cfg["weight_decay"])
    else:
        opt = torch.optim.AdamW(model.parameters(), lr=cfg["lr"], weight_decay=cfg["weight_decay"])
    loss_fn = nn.MSELoss()
    best = {"val": float("inf"), "state": None, "ep": -1, "curve": []}
    bad = 0
    for ep in range(CONFIG["EPOCHS"]):
        model.train()
        tr_sum, tr_n = 0.0, 0
        for xb, yb in train_loader:
            xb, yb = xb.to(dev).float(), yb.to(dev).float()
            opt.zero_grad()
            l = loss_fn(model(xb), yb)
            l.backward()
            if cfg["grad_clip_norm"]:
                nn.utils.clip_grad_norm_(model.parameters(), cfg["grad_clip_norm"])
            opt.step()
            tr_sum += l.item() * len(xb); tr_n += len(xb)
        model.eval()
        vs, vn = 0.0, 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(dev).float(), yb.to(dev).float()
                vs += loss_fn(model(xb), yb).item() * len(xb); vn += len(xb)
        v = vs / max(1, vn)
        best["curve"].append((ep, tr_sum / max(1, tr_n), v))
        if v < best["val"] - 1e-9:
            best["val"] = v
            best["state"] = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            best["ep"] = ep
            bad = 0
        else:
            bad += 1
            if bad >= cfg["patience"]:
                break
    epochs_run = ep + 1
    stopped_early = (bad >= cfg["patience"]) and (epochs_run < CONFIG["EPOCHS"])
    if best["state"] is not None:
        model.load_state_dict(best["state"])
    return model, best, {"epochs_run": int(epochs_run), "early_stop_triggered": bool(stopped_early), "best_epoch": int(best["ep"]), "best_val_loss": float(best["val"])}

def predict_tensor(model, X):
    dev = CONFIG["DEVICE"]
    model = model.to(dev); model.eval()
    out = []
    with torch.inference_mode():
        for i in range(0, len(X), 1024):
            xb = torch.from_numpy(X[i:i+1024]).to(dev).float()
            out.append(model(xb).cpu().numpy())
    return np.concatenate(out, axis=0)

# -------------------- metrics --------------------
def rmse(a, b): return float(np.sqrt(np.mean((a - b) ** 2)))
def mae(a, b):  return float(np.mean(np.abs(a - b)))

def theils_u2(y_true, y_hat, y_naive):
    den = rmse(y_true, y_naive)
    return float(rmse(y_true, y_hat) / (den if den > 1e-12 else 1.0))

def directional_accuracy_eps(y_true, y_hat, y_naive, eps):
    true_ret = (y_true - y_naive) / np.clip(y_naive, 1e-12, None)
    pred_ret = (y_hat - y_naive) / np.clip(y_naive, 1e-12, None)
    s_true = np.where(np.abs(true_ret) <= eps, 0, np.sign(true_ret))
    s_pred = np.where(np.abs(pred_ret) <= eps, 0, np.sign(pred_ret))
    mask = (s_true != 0)
    if not np.any(mask):
        return float("nan"), 0.0, 0
    return float(np.mean(s_true[mask] == s_pred[mask])), float(np.mean(mask)), int(np.sum(mask))

def pure_sign_trading_metrics(y_true, y_hat, y_naive, bps, verbose: bool = False):
    assert len(y_true) == len(y_hat) == len(y_naive), "Length mismatch in trading metrics."
    true_ret = (y_true - y_naive) / np.clip(y_naive, 1e-12, None)
    pred_ret = (y_hat - y_naive) / np.clip(y_naive, 1e-12, None)
    pos = np.sign(pred_ret).astype(float)  # {-1,0,+1}; exact ties -> 0

    # Turnover and per-change costs (clean assignment)
    if len(pos) > 1:
        delta = (np.abs(np.diff(pos)) > 0)
        turnover = int(np.sum(delta))
    else:
        delta = np.array([], dtype=bool)
        turnover = 0
    costs = np.zeros_like(pos, dtype=float)
    if len(pos) > 1:
        costs[1:] = delta.astype(float) * (bps / 10000.0)

    strategy_ret = pos * true_ret - costs
    mu = float(np.mean(strategy_ret))
    sd = float(np.std(strategy_ret, ddof=0))
    sharpe = float(np.sqrt(252.0) * mu / sd) if sd > 0 else 0.0
    curve = np.cumprod(1.0 + strategy_ret)
    if curve.size:
        peak = np.maximum.accumulate(curve); maxdd = float(np.max(1.0 - curve / peak))
    else:
        maxdd = 0.0
    if verbose:
        total_cost = (bps / 10000.0) * float(np.sum(delta))
        print(f"[Trading sanity] bps={bps:.1f} mean(ret)={np.mean(true_ret):.6g} std(ret)={np.std(true_ret):.6g} "
              f"n_changes={turnover} total_cost={total_cost:.6g}")
    return sharpe, maxdd, turnover

# -------------------- misc utils --------------------
def features_sha256(feats: List[str]) -> str:
    blob = "\n".join([str(x) for x in feats]).encode("utf-8")
    return hashlib.sha256(blob).hexdigest()

def save_training_curves_png(path: Path, curves: List[Tuple[int, float, float]], title: str = "", annotate: Optional[Dict[str, Any]] = None):
    fig = plt.figure(figsize=(6, 4))
    ep = [e for e, _, _ in curves]
    tr = [t for _, t, _ in curves]
    va = [v for _, _, v in curves]
    plt.plot(ep, tr, label="train"); plt.plot(ep, va, label="val")
    plt.xlabel("epoch"); plt.ylabel("MSE (original units)")
    if title: plt.title(title)
    plt.legend(); plt.tight_layout()
    if annotate:
        txt = f"best_ep={annotate.get('best_epoch')}, best_val={annotate.get('best_val_loss'):.4g}\n" \
              f"epochs_run={annotate.get('epochs_run')}, ES={annotate.get('early_stop_triggered')}"
        plt.gcf().text(0.99, 0.02, txt, ha="right", va="bottom", fontsize=8)
    fig.savefig(path, dpi=160); plt.close(fig)

def compute_param_count(n_in: int, cfg: Dict[str, Any]) -> int:
    m = LSTMReg(n_in=n_in, hidden=cfg["hidden_size"], layers=cfg["num_layers"], dropout=cfg["dropout"])
    return int(sum(p.numel() for p in m.parameters()))

# -------------------- tuning (per ticker, Val-only) --------------------
def run_search_for_ticker(ticker: str,
                          df_by_ticker: Dict[str, pd.DataFrame],
                          feat_cols: List[str],
                          sent_cols: List[str],
                          out_root: Path) -> Dict[str, Any]:
    full = df_by_ticker[ticker]
    tr = slice_dates(full, CONFIG["TRAIN_START"], CONFIG["TRAIN_END"])
    va = slice_dates(full, CONFIG["VAL_START"], CONFIG["VAL_END"])

    sc = ZeroPreservingStandardiser(sent_cols).fit(tr[feat_cols])
    tr_s = tr.copy(); va_s = va.copy()
    tr_s[feat_cols] = sc.transform(tr[feat_cols]); va_s[feat_cols] = sc.transform(va[feat_cols])

    Xtr, Ytr = windowise(tr_s, feat_cols, CONFIG["SEQ_LEN"])
    va_full = pd.concat([tr_s.tail(CONFIG["SEQ_LEN"]), va_s], ignore_index=True)
    Xva, Yva = windowise(va_full, feat_cols, CONFIG["SEQ_LEN"])

    # --- ESSENTIAL FIX: naïve last-close in ORIGINAL units (raw Close) ---
    va_full_close_raw = pd.concat([tr.tail(CONFIG["SEQ_LEN"])["Close"], va["Close"]], ignore_index=True)
    last_naive_va = naive_last_close_from_windowised(va_full_close_raw, CONFIG["SEQ_LEN"])
    # ---------------------------------------------------------------------

    rows, best, best_id = [], None, None
    search_seed = RANDOM_SEED

    for i in range(CONFIG["N_TRIALS"]):
        cfg = {
            "hidden_size": random.choice(CONFIG["SEARCH_SPACE"]["hidden_size"]),
            "num_layers": random.choice(CONFIG["SEARCH_SPACE"]["num_layers"]),
            "dropout": random.choice(CONFIG["SEARCH_SPACE"]["dropout"]),
            "optim": random.choice(CONFIG["SEARCH_SPACE"]["optim"]),
            "lr": random.choice(CONFIG["SEARCH_SPACE"]["lr"]),
            "batch_size": random.choice(CONFIG["SEARCH_SPACE"]["batch_size"]),
            "patience": random.choice(CONFIG["SEARCH_SPACE"]["patience"]),
            "weight_decay": random.choice(CONFIG["SEARCH_SPACE"]["weight_decay"]),
            "grad_clip_norm": random.choice(CONFIG["SEARCH_SPACE"]["grad_clip_norm"]),
            "seq_len": CONFIG["SEQ_LEN"]
        }
        tr_dl = build_loader(Xtr, Ytr, cfg["batch_size"])
        va_dl = build_loader(Xva, Yva, cfg["batch_size"])
        model = LSTMReg(n_in=len(feat_cols), hidden=cfg["hidden_size"],
                        layers=cfg["num_layers"], dropout=cfg["dropout"])
        model, best_stats, _ = train_one(model, tr_dl, va_dl, cfg)
        yv_hat = predict_tensor(model, Xva)

        v_rmse = rmse(Yva, yv_hat)
        v_mae  = mae(Yva, yv_hat)
        v_u2   = theils_u2(Yva, yv_hat, last_naive_va)

        rows.append({"trial": i, **cfg, "val_rmse": v_rmse, "val_mae": v_mae,
                     "val_u2": v_u2, "best_ep": best_stats["ep"], "best_val_loss": best_stats["val"]})

        better = (best is None) or (v_rmse < best["val_rmse"] - 1e-12) \
                 or (abs(v_rmse - best["val_rmse"]) < 1e-12 and (v_mae < best["val_mae"] - 1e-12))
        if better:
            best = {"cfg": cfg, "val_rmse": v_rmse, "val_mae": v_mae, "val_u2": v_u2, "curve": best_stats["curve"]}
            best_id = i

    out_dir = out_root / "hypertune" / ticker
    out_dir.mkdir(parents=True, exist_ok=True)
    log_path = out_dir / f"search_log_{ticker}.csv"
    pd.DataFrame(rows).sort_values(["val_rmse", "val_mae", "val_u2"]).to_csv(log_path, index=False)

    tuning_meta = {
        "ticker": ticker,
        "search_space": CONFIG["SEARCH_SPACE"],
        "n_trials": CONFIG["N_TRIALS"],
        "best_trial_id": int(best_id),
        "search_seed": int(search_seed),
        "val_objective": "RMSE",
        "tie_break": "MAE",
        "log_path": str(log_path.as_posix()),
        "best_val": {"rmse": float(best["val_rmse"]), "mae": float(best["val_mae"]), "u2": float(best["val_u2"])},
        "best_params": best["cfg"]
    }
    (out_dir / f"hypertune_summary_{ticker}.json").write_text(json.dumps(tuning_meta, indent=2), encoding="utf-8")

    # Optional: persist best curve PNG for audits
    curve_png = out_dir / f"best_curve_{ticker}.png"
    save_training_curves_png(curve_png, best["curve"], title=f"{ticker} hypertune best")

    return {"tuned_cfg": best["cfg"], "tuning_meta": tuning_meta}

# -------------------- monthly refit rollout (Test) --------------------
def monthly_refit_rollout(ticker, df_full, feat_cols, sent_cols, tuned_cfg, out_dir_t) -> Dict[str, Any]:
    L = tuned_cfg["seq_len"]
    tr = slice_dates(df_full, CONFIG["TRAIN_START"], CONFIG["TRAIN_END"])
    va = slice_dates(df_full, CONFIG["VAL_START"], CONFIG["VAL_END"])
    te = slice_dates(df_full, CONFIG["TEST_START"], CONFIG["TEST_END"])
    starts = monthly_boundaries(te)  # positional offsets

    all_dates, y_true, y_hat, y_naive, flags = [], [], [], [], []
    refit_dates, epochs_per_refit = [], []
    refit_es_rows = []
    param_count = None

    for ridx, start_pos in enumerate(starts):
        t0 = start_pos
        first_date = te.iloc[t0]["date"]

        # History strictly before day t (position-safe)
        hist = pd.concat([tr, va, te.iloc[:t0]], ignore_index=True)

        sc = ZeroPreservingStandardiser([c for c in feat_cols if c in sent_cols]).fit(hist[feat_cols])
        hist_s = hist.copy(); hist_s[feat_cols] = sc.transform(hist[feat_cols])

        # Per-refit zero-preservation evidence on history used for this refit
        zp_path = out_dir_t / f"zero_preservation_check_{ticker}_{pd.to_datetime(first_date).date()}.csv"
        zero_preservation_report(
            raw_df=hist,
            scaled_df=hist_s,
            sent_cols=[c for c in feat_cols if c in sent_cols],
            out_csv=zp_path
        )

        tr_s = slice_dates(hist_s, CONFIG["TRAIN_START"], CONFIG["TRAIN_END"])
        va_s = slice_dates(hist_s, CONFIG["VAL_START"], CONFIG["VAL_END"])

        Xtr, Ytr = windowise(tr_s, feat_cols, L)
        va_full = pd.concat([tr_s.tail(L), va_s], ignore_index=True)
        Xva, Yva = windowise(va_full, feat_cols, L)

        tr_dl = build_loader(Xtr, Ytr, tuned_cfg["batch_size"])
        va_dl = build_loader(Xva, Yva, tuned_cfg["batch_size"])

        model = LSTMReg(n_in=len(feat_cols), hidden=tuned_cfg["hidden_size"],
                        layers=tuned_cfg["num_layers"], dropout=tuned_cfg["dropout"])
        model, best, es = train_one(model, tr_dl, va_dl, tuned_cfg)

        if param_count is None:
            param_count = int(sum(p.numel() for p in model.parameters()))

        refit_dates.append(str(pd.to_datetime(first_date).date()))
        epochs_per_refit.append(int(es["epochs_run"]))
        refit_es_rows.append({
            "refit_index": int(ridx+1),
            "refit_date": str(pd.to_datetime(first_date).date()),
            "epochs_run": int(es["epochs_run"]),
            "best_epoch": int(es["best_epoch"]),
            "best_val_loss": float(es["best_val_loss"]),
            "early_stop_triggered": bool(es["early_stop_triggered"])
        })

        # Prepare Test chunk features (position-safe slicing)
        end_pos = (len(te) if (ridx + 1) == len(starts) else starts[ridx + 1])
        te_chunk = te.iloc[t0:end_pos]
        ctx = pd.concat([va.tail(L), te_chunk], ignore_index=True)
        ctx_scaled = sc.transform(ctx[feat_cols])
        te_feat_scaled = ctx_scaled.iloc[L:].reset_index(drop=True)
        te_scaled = te_chunk.reset_index(drop=True).copy()
        te_scaled[feat_cols] = te_feat_scaled

        df_for_test = pd.concat([va.tail(L).reset_index(drop=True), te_scaled], ignore_index=True)
        Xte, Yte = windowise(df_for_test, feat_cols, L)

        # naïve last-close aligned: Close_{t-1} for each Yte (original units)
        close_ctx = pd.concat([va.tail(L)["Close"], te_chunk["Close"]], ignore_index=True)
        lc = close_ctx.to_numpy()[L-1 : L-1 + len(Yte)]

        yh = predict_tensor(model, Xte)
        n_local = len(Yte)
        dates_local = te_chunk["date"].reset_index(drop=True).iloc[:n_local]
        all_dates.append(dates_local)
        y_true.append(Yte); y_hat.append(yh); y_naive.append(lc)
        flags.append(np.zeros(n_local, dtype=bool))

        if ridx == 0:
            save_training_curves_png(out_dir_t / f"training_curves_{CONFIG['MODEL_ID']}_{ticker}.png",
                                     best["curve"],
                                     title=f"{ticker} best refit",
                                     annotate=es)

    # Self-check: number of refits equals number of Test months
    expected_months = int(te["date"].dt.to_period("M").nunique())
    if len(refit_dates) != expected_months:
        raise RuntimeError(f"[{ticker}] refit count {len(refit_dates)} != expected Test months {expected_months}")

    # Persist ES per-refit table
    pd.DataFrame(refit_es_rows).to_csv(out_dir_t / f"early_stopping_{CONFIG['MODEL_ID']}_{ticker}.csv", index=False)

    dates = pd.concat(all_dates, ignore_index=True)
    y_true = np.concatenate(y_true); y_hat = np.concatenate(y_hat); y_naive = np.concatenate(y_naive)

    # Assertions
    assert len(y_true) == CONFIG["ASSERT_N_TEST"] == len(y_hat), "Test length assertion failed."
    for name, arr in {"y_true": y_true, "y_hat": y_hat, "y_naive": y_naive}.items():
        assert np.all(np.isfinite(arr)), f"Non-finite values in {name}."

    # Predictions CSV
    pred_df = pd.DataFrame({
        "date": dates,
        "y_true": y_true,
        "y_hat": y_hat,
        "residual": y_true - y_hat,
        "in_sample_flag": np.concatenate(flags)
    })
    pred_df.to_csv(out_dir_t / f"predictions_{CONFIG['MODEL_ID']}_{ticker}.csv", index=False)

    # Metrics (numeric only)
    da, cov, n_da = directional_accuracy_eps(y_true, y_hat, y_naive, CONFIG["EPS_DA"])
    rmse_v = rmse(y_true, y_hat)
    mae_v  = mae(y_true, y_hat)
    u2_v   = theils_u2(y_true, y_hat, y_naive)
    sh0,  dd0,  turn = pure_sign_trading_metrics(y_true, y_hat, y_naive, bps=0.0,  verbose=False)
    sh10, dd10, _    = pure_sign_trading_metrics(y_true, y_hat, y_naive, bps=10.0, verbose=False)

    metrics = {
        "Ticker": ticker,
        "RMSE": float(rmse_v),
        "MAE": float(mae_v),
        "U2": float(u2_v),
        "DA_epsilon": float(da) if np.isfinite(da) else 0.0,
        "Coverage": float(cov),
        "n": int(len(y_true)),
        "n_da": int(n_da),
        "Sharpe_0bps": float(sh0),
        "Sharpe_10bps": float(sh10),
        "MaxDD_0bps": float(dd0),
        "MaxDD_10bps": float(dd10),
        "Turnover": int(turn)
    }
    Path(out_dir_t / f"metrics_{CONFIG['MODEL_ID']}_{ticker}.json").write_text(json.dumps(metrics, indent=2), encoding="utf-8")

    run_cfg = {
        "reported_model_name": CONFIG["REPORTED_NAME"],
        "model_id": CONFIG["MODEL_ID"],
        "ticker": ticker,
        "splits": {"train": [CONFIG["TRAIN_START"], CONFIG["TRAIN_END"]],
                   "val": [CONFIG["VAL_START"], CONFIG["VAL_END"]],
                   "test": [CONFIG["TEST_START"], CONFIG["TEST_END"]],
                   "n_test": CONFIG["ASSERT_N_TEST"]},
        "cadence": "monthly_refit",
        "refit_dates": refit_dates,
        "epochs_per_refit": epochs_per_refit,  # epochs actually run
        "early_stopping": {
            "policy": {"patience": int(tuned_cfg["patience"])},
            "per_refit": refit_es_rows
        },
        "parameter_count": int(param_count) if param_count is not None else None,
        "seeds": {"python": RANDOM_SEED, "numpy": RANDOM_SEED, "torch": RANDOM_SEED,
                  "cuda_seed": RANDOM_SEED if torch.cuda.is_available() else None},
        "window_length": CONFIG["SEQ_LEN"],
        "batch_size": tuned_cfg["batch_size"],
        "tuned_hyperparameters": {k: v for k, v in tuned_cfg.items()},
        "target_scaling": "none",
        "inverse_transform_applied": False,
        "features_used": feat_cols,
        "features_sha256": features_sha256(feat_cols),
        "sentiment_zero_columns": [c for c in feat_cols if c in sent_cols],
        "clock_invariants": CONFIG["CLOCK_INVARIANTS"],
        "tuning_meta": {},  # set by caller
        "device": CONFIG["DEVICE"]
    }
    Path(out_dir_t / f"run_config_{CONFIG['MODEL_ID']}_{ticker}.json").write_text(json.dumps(run_cfg, indent=2), encoding="utf-8")
    return metrics

# -------------------- manifests / packaging --------------------
APACHE_2_TEXT = """Apache License
Version 2.0, January 2004
http://www.apache.org/licenses/
(Full text should be included in production)"""

def write_env_manifest(run_root: Path):
    env = {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "torch": torch.__version__,
        "numpy": np.__version__,
        "pandas": pd.__version__,
        "device": CONFIG["DEVICE"],
        "timestamp_utc": datetime.utcnow().isoformat()
    }
    (run_root / "env_manifest.txt").write_text("\n".join([f"{k}: {v}" for k, v in env.items()]) + "\n", encoding="utf-8")

def write_file_hashes(run_root: Path):
    rows = []
    for r, _, fs in os.walk(run_root):
        for fn in fs:
            p = Path(r) / fn
            h = hashlib.sha256(p.read_bytes()).hexdigest()
            rows.append({"path": str(p.relative_to(run_root)).replace("\\", "/"),
                         "sha256": h, "size": p.stat().st_size})
    (run_root / "file_hashes.json").write_text(json.dumps(rows, indent=2), encoding="utf-8")

def write_provenance(run_root: Path):
    prov = [CONFIG["PROVENANCE"]]
    (run_root / "code_provenance.csv").write_text(pd.DataFrame(prov).to_csv(index=False), encoding="utf-8")
    tpd = run_root / "third_party_licenses"; tpd.mkdir(parents=True, exist_ok=True)
    (tpd / "Apache-2.0.txt").write_text(APACHE_2_TEXT, encoding="utf-8")
    (tpd / "README.txt").write_text(
        f"Keras-io example adapted\nCommit: {CONFIG['PROVENANCE']['commit_sha']}\nLicence: {CONFIG['PROVENANCE']['licence']}\n",
        encoding="utf-8"
    )

def write_outputs_index(run_root: Path):
    root = run_root / CONFIG["MODEL_ID"]
    idx = {}
    for t in CONFIG["TICKERS"]:
        d = root / t
        idx[t] = sorted([p.name for p in d.glob("*")]) if d.exists() else []
    (run_root / "outputs_index.json").write_text(json.dumps(idx, indent=2), encoding="utf-8")

def write_readme(run_root: Path, feats_used: List[str], tuned_cfg_map: Dict[str, Dict[str, Any]], typical_param_count: Optional[int]):
    txt = []
    txt.append(f"{CONFIG['MODEL_ID']} pack"); txt.append("=" * len(txt[-1])); txt.append("")
    txt.append(f"Model: {CONFIG['REPORTED_NAME']}"); txt.append(f"Tickers: {', '.join(CONFIG['TICKERS'])}"); txt.append("")
    txt.append("Splits:")
    txt.append(f"  - Train: {CONFIG['TRAIN_START']} to {CONFIG['TRAIN_END']}")
    txt.append(f"  - Val:   {CONFIG['VAL_START']} to {CONFIG['VAL_END']}")
    txt.append(f"  - Test:  {CONFIG['TEST_START']} to {CONFIG['TEST_END']} (n={CONFIG['ASSERT_N_TEST']})")
    txt.append("")
    txt.append("Trading diagnostics: pure-sign; costs on position changes; Turnover counts position changes.")
    txt.append(f"Epsilon for DA: {CONFIG['EPS_DA']}")
    if typical_param_count is not None:
        txt.append(f"Typical parameter count: ~{typical_param_count}")
    txt.append("")
    txt.append(f"Features parity hash: {features_sha256(feats_used)}"); txt.append("")
    txt.append("Tuned hyperparameters (per ticker):")
    for t in CONFIG["TICKERS"]:
        txt.append(f"  - {t}: {json.dumps(tuned_cfg_map[t], separators=(',',':'))}")
    txt.append(""); txt.append("Top-level artefacts:")
    txt += [
        "  features_manifest.json",
        "  cross_model_features_parity_stub.json",
        "  cross_model_features_parity_report.json",
        "  env_manifest.txt",
        "  file_hashes.json",
        "  code_provenance.csv",
        "  outputs_index.json",
        "  hypertune/<TICKER>/hypertune_summary_<TICKER>.json",
        "  hypertune/<TICKER>/search_log_<TICKER>.csv",
        "  README.txt",
        "  third_party_licenses/"
    ]
    (run_root / "README.txt").write_text("\n".join(txt) + "\n", encoding="utf-8")

def assert_checklist_files(run_root: Path):
    root = run_root / CONFIG["MODEL_ID"]
    required_top = [
        run_root / "features_manifest.json",
        run_root / "cross_model_features_parity_stub.json",
        run_root / "cross_model_features_parity_report.json",
        run_root / "env_manifest.txt",
        run_root / "file_hashes.json",
        run_root / "code_provenance.csv",
        run_root / "outputs_index.json",
        run_root / "README.txt",
    ]
    for p in required_top:
        if not p.exists():
            raise RuntimeError(f"Missing top-level artefact: {p}")
    # per-ticker hypertune artefacts
    for t in CONFIG["TICKERS"]:
        ht = run_root / "hypertune" / t
        if not (ht / f"hypertune_summary_{t}.json").exists(): raise RuntimeError(f"Missing hypertune_summary for {t}")
        if not (ht / f"search_log_{t}.csv").exists(): raise RuntimeError(f"Missing search_log for {t}")
    # per-ticker outputs
    for t in CONFIG["TICKERS"]:
        d = root / t
        if not d.exists():
            raise RuntimeError(f"Missing ticker directory: {d}")
        req = {
            "predictions": d / f"predictions_{CONFIG['MODEL_ID']}_{t}.csv",
            "metrics":     d / f"metrics_{CONFIG['MODEL_ID']}_{t}.json",
            "run_config":  d / f"run_config_{CONFIG['MODEL_ID']}_{t}.json",
            "curves":      d / f"training_curves_{CONFIG['MODEL_ID']}_{t}.png",
            "zero_check_any":  d.glob(f"zero_preservation_check_{t}_*.csv"),  # at least one per-refit file
            "es_table":    d / f"early_stopping_{CONFIG['MODEL_ID']}_{t}.csv",
        }
        for key in ["predictions","metrics","run_config","curves","es_table"]:
            pth = req[key]
            if not pth.exists():
                raise RuntimeError(f"[{t}] missing {key}: {pth}")
        # ensure at least one per-refit zero-pres file exists
        if not any(req["zero_check_any"]):
            raise RuntimeError(f"[{t}] missing per-refit zero-preservation reports")
        n_pred = sum(1 for _ in open(req["predictions"], "r", encoding="utf-8")) - 1
        if n_pred != CONFIG["ASSERT_N_TEST"]:
            raise RuntimeError(f"[{t}] predictions rows={n_pred}, expected {CONFIG['ASSERT_N_TEST']}")
        m = json.loads((d / f"metrics_{CONFIG['MODEL_ID']}_{t}.json").read_text(encoding="utf-8"))
        if m.get("n", None) != CONFIG["ASSERT_N_TEST"]:
            raise RuntimeError(f"[{t}] metrics 'n' must be {CONFIG['ASSERT_N_TEST']}, got {m.get('n')}")

def write_features_parity(out_root: Path, feats_used: List[str]):
    sha = features_sha256(feats_used)
    (out_root / "features_manifest.json").write_text(
        json.dumps({
            "features_used": feats_used,
            "features_sha256": sha,
            "sentiment_zero_columns": [c for c in feats_used if c.startswith(("Tw_","Rd_","Nw_SP500_"))],
            "intended_cross_model_parity": "Compare this SHA256 across packs to evidence parity."
        }, indent=2), encoding="utf-8"
    )
    (out_root / "cross_model_features_parity_stub.json").write_text(
        json.dumps({"model_id": CONFIG["MODEL_ID"], "features_sha256": sha}, indent=2), encoding="utf-8"
    )

    report = {"this_model": {"model_id": CONFIG["MODEL_ID"], "features_sha256": sha, "count": len(feats_used)}}
    ident_flags = []

    # 1) Try sibling packs
    peers_dirs = {
        "TRANSFORMER": Path("TRANSFORMER_FINAL") / "TRANSFORMER",
        "HYBRID": Path("HYBRID_FINAL") / "HYBRID",
        "LSTM_PO": Path("LSTM_PO_FINAL") / "LSTM_PO"
    }
    for name, base in peers_dirs.items():
        try:
            run_cfg_path = next(base.rglob("run_config_*_*.json"))
            peer_feats = json.loads(run_cfg_path.read_text(encoding="utf-8")).get("features_used", [])
            peer_sha = features_sha256(peer_feats) if peer_feats else ""
            same = (peer_sha == sha)
            ident_flags.append(same)
            report[name] = {"features_sha256": peer_sha, "count": len(peer_feats), "identical_to_this": bool(same), "source": "pack"}
        except StopIteration:
            pass
        except Exception as e:
            report[name] = {"features_sha256": "", "count": 0, "identical_to_this": False, "source": "pack_error", "note": str(e)}

    # 2) Optional signed parity via file
    pfile = Path(CONFIG["PEER_FEATURES_FILE"])
    if pfile.exists():
        try:
            peer_map = json.loads(pfile.read_text(encoding="utf-8"))
            for name, feats in peer_map.items():
                peer_sha = features_sha256(feats) if feats else ""
                same = (peer_sha == sha)
                ident_flags.append(same)
                report[name] = {"features_sha256": peer_sha, "count": len(feats), "identical_to_this": bool(same), "source": "manifest"}
        except Exception as e:
            report["peer_manifest_error"] = str(e)

    report["all_identical"] = bool(ident_flags and all(ident_flags))
    (out_root / "cross_model_features_parity_report.json").write_text(json.dumps(report, indent=2), encoding="utf-8")

def build_zip(run_root: Path, outfile: Optional[Path] = None) -> Path:
    run_root = run_root.resolve()
    if outfile is None:
        stamp = time.strftime("%Y%m%d_%H%M%S", time.gmtime())
        outfile = run_root.parent / f"{run_root.name}_pack_{stamp}.zip"
    with zipfile.ZipFile(outfile, "w", compression=zipfile.ZIP_DEFLATED, allowZip64=True) as zf:
        for p in sorted(run_root.rglob("*")):
            if p.is_dir(): continue
            arc = str(p.relative_to(run_root)).replace("\\", "/")
            zf.write(p, arcname=arc)
    h = hashlib.sha256(outfile.read_bytes()).hexdigest()
    (run_root / "bundle_sha256.txt").write_text(f"{outfile.name}  {h}\n", encoding="utf-8")
    print("Pack created:", outfile)
    print("SHA256:", (run_root / "bundle_sha256.txt").read_text(encoding="utf-8").strip())
    return outfile

# -------------------- main --------------------
def main():
    OUT_ROOT = Path(CONFIG["OUT_ROOT"])
    MODEL_DIR = OUT_ROOT / CONFIG["MODEL_ID"]
    OUT_ROOT.mkdir(parents=True, exist_ok=True); MODEL_DIR.mkdir(parents=True, exist_ok=True)

    # Load data
    df_by_t = {t: read_csv(t) for t in CONFIG["TICKERS"]}
    train_by_t = {t: slice_dates(df_by_t[t], CONFIG["TRAIN_START"], CONFIG["TRAIN_END"]) for t in CONFIG["TICKERS"]}
    val_by_t   = {t: slice_dates(df_by_t[t], CONFIG["VAL_START"], CONFIG["VAL_END"]) for t in CONFIG["TICKERS"]}

    # Features and sentiment groups (stable across tickers)
    any_df = df_by_t[CONFIG["TICKERS"][0]]
    _, sent_all = group_cols(list(any_df.columns))
    feat_cols_use = feature_stable_set(train_by_t, val_by_t)
    sent_cols_use = [c for c in feat_cols_use if c in sent_all]

    # Critical column and feature presence checks
    required = {"Target", "Close", "date"}
    missing = required - set(any_df.columns)
    if missing:
        raise RuntimeError(f"Input is missing required columns: {sorted(missing)}")
    if "Close" not in feat_cols_use:
        raise RuntimeError("`Close` must be in features for naïve baseline alignment.")
    for t in CONFIG["TICKERS"]:
        cols_missing = set(feat_cols_use) - set(df_by_t[t].columns)
        if cols_missing:
            raise RuntimeError(f"{t}: missing feature columns: {sorted(cols_missing)}")

    # Per-ticker validation-only search
    tuned_cfg_map: Dict[str, Dict[str, Any]] = {}
    tuning_meta_map: Dict[str, Dict[str, Any]] = {}
    for t in CONFIG["TICKERS"]:
        res = run_search_for_ticker(t, df_by_t, feat_cols_use, sent_cols_use, OUT_ROOT)
        tuned_cfg_map[t] = res["tuned_cfg"]
        tuning_meta_map[t] = res["tuning_meta"]

    # Compute a typical parameter count (same across tickers given same n_in & hyperparams)
    typical_param_count = compute_param_count(len(feat_cols_use), tuned_cfg_map[CONFIG["TICKERS"][0]])

    # Monthly refit rollout for all tickers with their tuned config
    rows = []
    for t in CONFIG["TICKERS"]:
        tdir = MODEL_DIR / t
        tdir.mkdir(parents=True, exist_ok=True)
        m = monthly_refit_rollout(t, df_by_t[t], feat_cols_use, sent_cols_use, tuned_cfg_map[t], tdir)
        rows.append(m)
        # Inject per-ticker tuning meta into run_config
        rc_path = tdir / f"run_config_{CONFIG['MODEL_ID']}_{t}.json"
        rc = json.loads(rc_path.read_text(encoding="utf-8"))
        rc["tuning_meta"] = tuning_meta_map[t]
        Path(rc_path).write_text(json.dumps(rc, indent=2), encoding="utf-8")

    # Summary table
    table = pd.DataFrame(rows)[["Ticker","RMSE","MAE","U2","DA_epsilon","Coverage","n","n_da",
                                "Sharpe_0bps","Sharpe_10bps","MaxDD_0bps","MaxDD_10bps","Turnover"]]
    table.to_csv(OUT_ROOT / "tuned_test_table.csv", index=False)

    # Features parity
    write_features_parity(OUT_ROOT, feat_cols_use)

    # Provenance and manifests
    write_provenance(OUT_ROOT)
    write_env_manifest(OUT_ROOT)
    write_file_hashes(OUT_ROOT)
    write_outputs_index(OUT_ROOT)
    write_readme(OUT_ROOT, feat_cols_use, tuned_cfg_map, typical_param_count)

    # Final checklist and packaging
    assert_checklist_files(OUT_ROOT)
    build_zip(OUT_ROOT)

if __name__ == "__main__":
    main()

Pack created: /content/LSTM_SE_TUNED_FINAL_pack_20250930_124037.zip
SHA256: LSTM_SE_TUNED_FINAL_pack_20250930_124037.zip  d71dbd07dbbd15842614432feea7137285e4e79b8db2414f7c47dd25b3e8a4fa


In [ ]:
from google.colab import files
import glob, os

# find the most recent pack produced by build_zip(...)
zips = sorted(glob.glob("LSTM_SE_TUNED_FINAL_pack_*.zip"), key=os.path.getmtime)
if not zips:
    raise FileNotFoundError("No ZIP found in the working directory.")
files.download(zips[-1])  # triggers a browser download

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>